In [1]:
import pandas as pd
import numpy as np

X = pd.read_pickle(f"../input.pkl")
y = pd.read_pickle(f"../output.pkl")

In [2]:
X.columns

Index(['gender', 'age', 'race', 'educ', 'marry', 'house', 'pov', 'wt', 'ht',
       'bmi', 'wst', 'hip', 'dia', 'pulse', 'sys', 'alt', 'albumin', 'ast',
       'crea', 'chol', 'tyg', 'ggt', 'wbc', 'hb', 'hct', 'ldl', 'hdl',
       'acratio', 'glu', 'insulin', 'crp', 'hb1ac', 'mvpa', 'ac_week'],
      dtype='object')

In [3]:
df_clean=pd.concat([X,y],axis=1)

In [4]:
df_clean

,gender,age,race,educ,marry,house,pov,wt,ht,bmi,...,ldl,hdl,acratio,glu,insulin,crp,hb1ac,mvpa,ac_week,dbs
0,1.0,66.0,3.0,5.0,1.0,2.0,5.00,101.8,174.2,33.5,...,135.0,60.0,6.64,99.0,19.91,2.03,5.6,450.000000,17.307692,0.0
1,1.0,34.0,1.0,4.0,1.0,3.0,1.33,90.6,173.3,30.2,...,111.0,46.0,4.07,100.0,11.38,1.05,5.1,43.808976,4.000000,0.0
2,1.0,51.0,3.0,5.0,1.0,4.0,5.00,76.7,177.3,24.4,...,120.0,48.0,6.29,88.0,7.20,0.92,4.8,555.000000,0.576923,0.0
3,1.0,47.0,1.0,3.0,1.0,2.0,1.67,73.1,159.5,28.7,...,122.0,41.0,12.41,107.0,9.81,5.22,5.8,844.615385,3.500000,0.0
4,1.0,50.0,3.0,5.0,1.0,2.0,5.00,94.0,171.7,31.9,...,65.0,49.0,4.94,104.0,7.26,0.71,5.3,1260.000000,2.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758,1.0,68.0,3.0,4.0,1.0,4.0,1.30,94.8,178.6,29.7,...,150.0,47.0,46.56,99.0,16.61,3.69,5.4,1540.000000,0.173077,0.0
759,2.0,70.0,3.0,5.0,2.0,1.0,5.00,42.8,155.4,17.7,...,133.0,81.0,8.00,82.0,1.35,0.66,5.4,260.000000,1.000000,0.0
760,2.0,74.0,3.0,5.0,2.0,1.0,2.74,68.5,158.5,27.3,...,83.0,74.0,7.29,97.0,9.31,0.67,5.4,447.692308,4.000000,0.0
761,1.0,57.0,3.0,5.0,1.0,3.0,4.12,74.4,176.9,23.8,...,171.0,55.0,4.55,105.0,6.60,0.42,5.2,300.000000,7.000000,0.0


In [5]:
df_clean.loc[762,:]

gender       1.000000
age         33.000000
race         1.000000
educ         4.000000
marry        1.000000
house        2.000000
pov          3.280000
wt         113.800000
ht         178.500000
bmi         35.700000
wst        121.800000
hip        113.300000
dia         79.000000
pulse       73.000000
sys        114.000000
alt         62.000000
albumin      4.000000
ast         32.000000
crea         0.820000
chol       170.000000
tyg        168.000000
ggt         36.000000
wbc          6.700000
hb          15.500000
hct         46.300000
ldl        106.000000
hdl         33.000000
acratio      5.240000
glu        204.000000
insulin     22.770000
crp          8.810000
hb1ac        9.500000
mvpa       840.000000
ac_week      0.173077
dbs          2.000000
Name: 762, dtype: float64

In [6]:
import torch.cuda
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


True
NVIDIA GeForce RTX 4070 SUPER


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader,Subset
from lightning.pytorch import LightningModule,Trainer
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold
from lightning.pytorch.loggers import CSVLogger
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


# 머신러닝 관련 라이브러리
from sklearn.metrics import classification_report, confusion_matrix

#기억력 관련 변수 ->cerad 4,6,7,8(8,10,11,12)
#사고력 관련 변수 ->cerad 1,2,3,5,9,10,11,12 (5,6,7,9,13,14,15,16)
# 나머지 변수 -> age,gender, edu,s_adl,s_iadl  (0,1,2,3,4)
# 범주형 파생변수 -> j1v, 2,5,6,9,10,11,adl,iadl


def to_tensor(*dfs):
    return [torch.tensor(df.values,dtype=torch.float32) for df in dfs]



class MainModule(nn.Module):
    def __init__(self,input_dim=6,output_dim=3,do1=0.5,activ=0):
        super().__init__()
        self.activ=nn.GELU() if activ==1 else nn.ReLU()
        self.input_layer=nn.Sequential(
            nn.Linear(input_dim,6),
            self.activ,
            nn.Dropout(do1),
            
            nn.Linear(6,4),
            self.activ,
            nn.Dropout(do1),

            nn.Linear(4,3),
            self.activ,
            nn.Dropout(do1),
        )
        
        self.output_layer=nn.Linear(3,output_dim)
    def forward(self,x):
        x=self.input_layer(x)
        output=self.output_layer(x)
        return output

class SubModule(nn.Module):
    def __init__(self,input_dim=28,output_dim=7,do1=0.5,activ=0):
        super().__init__()
        self.activ=nn.GELU() if activ==1 else nn.ReLU()
        self.input_layer=nn.Sequential(
            nn.Linear(input_dim,28),
            self.activ,
            nn.Dropout(do1),
            
            nn.Linear(28,14),
            self.activ,
            nn.Dropout(do1),

            nn.Linear(14,7),
            self.activ,
            nn.Dropout(do1),
        )
        
        self.output_layer=nn.Linear(7,output_dim)
    def forward(self,x):
        x=self.input_layer(x)
        output=self.output_layer(x)
        return output

class TotalModel(nn.Module):    #TotalModel의 128 dim에서 skip connection 적용
    def __init__(self,do1,do2,activ):
        super().__init__()

        self.activ=nn.GELU() if activ==1 else nn.ReLU()

        self.module1=SubModule(6,3,do1,activ)
        self.module2=SubModule(28,7,do1,activ)

        
        self.hidden_layers=nn.ModuleList([
            nn.Sequential(
                nn.Linear(10,10),
                self.activ,
                nn.Dropout(do2),
            )for _ in range(1)
        ])
        
        self.output_layers=nn.Sequential(
            nn.Linear(10,5),
            self.activ,
            nn.Dropout(do2),
            
            nn.Linear(5,5),
            self.activ,
            nn.Dropout(do2),            
            
            nn.Linear(5,3)         
        )
        
    
    def forward(self,x):
        x1=x[:,[0,1,2,4,28,31]]
        x2=x[:,[3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,29,30,32,33]]

        out1=self.module1(x1)        #특정 4개의 열 데이터는 module1에 통과 -> output1 나온다      -shape:[batch_size,16]
        out2=self.module2(x2)        #나머지 13개의 열 데이터는 module2에 통과 -> output2 나온다   -shape:[batch_size,16]

        out=torch.cat([out1,out2],dim=1)       #-shape:[batch_size,32]
        k=out
        residual=k          #초기 결과를 k로 잔차 저장
        for layer in self.hidden_layers:
            j=layer(k)
            k=j+residual
            
        output=self.output_layers(k)
        return output
        


In [8]:

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)
print(y.value_counts())

scalar=StandardScaler()
X_train_pr=scalar.fit_transform(X_train)
X_test_pr=scalar.transform(X_test)

X_trainp = pd.DataFrame(X_train_pr, columns=X_train.columns, index=X_train.index)
X_testp = pd.DataFrame(X_test_pr, columns=X_test.columns, index=X_test.index)

dbs
0.0    701
2.0     44
1.0     18
Name: count, dtype: int64


In [9]:
import random
import numpy as np
import torch

def set_seed(seed=30):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


In [10]:
from torch.utils.data import TensorDataset, DataLoader
import copy


X_train_t = torch.tensor(X_trainp.to_numpy(), dtype=torch.float32)
y_train_t = torch.tensor(y_train.values, dtype=torch.long).squeeze()
y_test_t = torch.tensor(y_test.values, dtype=torch.long).squeeze()

X_test_t = torch.tensor(X_testp.to_numpy(), dtype=torch.float32)

train_ds = TensorDataset(X_train_t, y_train_t)
test_ds = TensorDataset(X_test_t, y_test_t)

train_loader = DataLoader(train_ds, batch_size=10, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=10, shuffle=False)

model=TotalModel(0.35,0.35,1).to(device)
criterion = nn.CrossEntropyLoss()   # logits → sigmoid + BCE
optimizer = optim.Adam(model.parameters(), lr=1e-3)

best_acc = 0.0
best_state_dict = None


def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0

    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        logits = model(X_batch)           # [B, 1]
        loss = criterion(logits, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * X_batch.size(0)

    return total_loss / len(loader.dataset)


def evaluate(model, loader):
    model.eval()
    preds = []
    trues = []

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            logits = model(X_batch)              # [B, 3] 
            prob = torch.softmax(logits, dim=1)  # [B, 3]
            pred = torch.argmax(prob, dim=1)     # [B]

            preds += pred.cpu().tolist()
            trues += y_batch.cpu().tolist()


    acc = accuracy_score(trues, preds)
    prec = precision_score(trues, preds,average='weighted')
    rec=recall_score(trues,preds,average='weighted')
    f1=f1_score(trues,preds,average='weighted')
    return acc,prec,rec,f1


epochs = 100

for epoch in range(epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    test_acc,prec,rec,f1 = evaluate(model, test_loader)

    print(f"Epoch {epoch+1}/{epochs} | Loss: {train_loss:.4f} | Test Acc: {test_acc:.4f}")
    if test_acc > best_acc:
        best_acc = test_acc
        best_state_dict = copy.deepcopy(model.state_dict())


c:\Users\cmc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\cmc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Epoch 1/100 | Loss: 1.1383 | Test Acc: 0.0654
Epoch 2/100 | Loss: 0.8593 | Test Acc: 0.9281


c:\Users\cmc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\cmc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Epoch 3/100 | Loss: 0.6336 | Test Acc: 0.9281
Epoch 4/100 | Loss: 0.5987 | Test Acc: 0.9281


c:\Users\cmc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\cmc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


...

c:\Users\cmc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\cmc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Epoch 95/100 | Loss: 0.1333 | Test Acc: 0.9412
Epoch 96/100 | Loss: 0.1393 | Test Acc: 0.9412


c:\Users\cmc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\cmc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Epoch 97/100 | Loss: 0.1383 | Test Acc: 0.9412
Epoch 98/100 | Loss: 0.1334 | Test Acc: 0.9412
Epoch 99/100 | Loss: 0.1577 | Test Acc: 0.9412
Epoch 100/100 | Loss: 0.1723 | Test Acc: 0.9412


c:\Users\cmc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\cmc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [11]:
model.load_state_dict(best_state_dict)
print("Best model restored. Best Acc =", best_acc)
print(evaluate(model,test_loader))


Best model restored. Best Acc = 0.954248366013072
(0.954248366013072, 0.9446889867057934, 0.954248366013072, 0.9464742067530587)


c:\Users\cmc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [12]:
print(X_trainp.shape)
print(X_testp.shape)


(610, 34)
(153, 34)


In [13]:
y_test_t

tensor([0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 2, 2, 0, 0])

In [14]:
model.eval()
scores = []

with torch.no_grad():
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)

        logits = model(X_batch)                # [B, 3]
        prob = torch.softmax(logits, dim=1)    # [B, 3]

        p0 = prob[:, 0]
        p1 = prob[:, 1]
        p2 = prob[:, 2]

        score = p1 + 2*p2
        scores.extend(score.cpu().numpy())
scores


[0.06805272,
 0.015110992,
 0.117255084,
 0.11021721,
 0.04796549,
 0.23620282,
 0.07396789,
 0.0780038,
 0.036567867,
 0.017220605,
 0.5979649,
 0.03161551,
 0.20547655,
 0.018215094,
 0.016569702,
 0.042714484,
 0.022436738,
 0.049025625,
 0.30031407,
 0.08604112,
 0.024491282,
 0.039350122,
 0.026133545,
 0.021278918,
 0.033109277,
 0.1628503,
 0.0316842,
 0.027801387,
 0.029245265,
 0.10962173,
 0.04799857,
 0.03166773,
 0.059680473,
 0.018085647,
 0.024661714,
 0.018423468,
 0.012425575,
 0.017471954,
 0.8013531,
 0.03922244,
 0.018822988,
 0.29232082,
 0.062059596,
 0.45259497,
 0.097089216,
 0.030266851,
 0.028008692,
 0.05213935,
 0.41182274,
 0.045989797,
 0.05397024,
 0.008544876,
 0.021182217,
 0.038410336,
 0.012371425,
 0.024057481,
 0.19164628,
 0.029956654,
 0.15837541,
 0.026665352,
 0.03242196,
 0.009839685,
 0.02537693,
 0.05265516,
 0.06813819,
 0.018892132,
 0.09201218,
 0.07168379,
 1.5459845,
 0.05613754,
 0.047965158,
 0.18200216,
 0.037452128,
 0.62191486,
 0.07

In [15]:
X_test_t


tensor([[ 0.9902,  0.6613, -0.1470,  ...,  0.0353,  0.0910, -0.3949],
        [ 0.9902, -1.0258,  0.6327,  ..., -0.6970,  1.9700, -0.3949],
        [-1.0099,  1.5652, -0.1470,  ...,  0.3282,  0.1109,  0.3673],
        ...,
        [ 0.9902,  0.5408,  0.6327,  ...,  2.2322,  0.7173, -0.6646],
        [ 0.9902, -1.1463,  0.6327,  ...,  0.0353, -0.8621, -0.0900],
        [-1.0099,  0.1793,  2.1922,  ...,  0.9141, -1.2297, -0.5239]])

In [16]:
model.eval()
model.to(device)
X_test_t=X_test_t.to(device)
output=model(X_test_t)
prob=torch.softmax(output,dim=1)
prob


tensor([[0.9536, 0.0247, 0.0217],
        [0.9884, 0.0081, 0.0035],
        [0.9232, 0.0363, 0.0405],
        [0.9275, 0.0347, 0.0378],
        [0.9664, 0.0191, 0.0144],
        [0.8524, 0.0589, 0.0886],
        [0.9499, 0.0261, 0.0239],
        [0.9474, 0.0272, 0.0254],
        [0.9739, 0.0157, 0.0104],
        [0.9869, 0.0090, 0.0041],
        [0.6431, 0.1159, 0.2410],
        [0.9771, 0.0141, 0.0088],
        [0.8705, 0.0535, 0.0760],
        [0.9862, 0.0093, 0.0044],
        [0.9874, 0.0087, 0.0039],
        [0.9699, 0.0176, 0.0126],
        [0.9833, 0.0109, 0.0058],
        [0.9658, 0.0194, 0.0148],
        [0.8151, 0.0694, 0.1155],
        [0.9424, 0.0291, 0.0285],
        [0.9819, 0.0117, 0.0064],
        [0.9720, 0.0166, 0.0114],
        [0.9808, 0.0122, 0.0069],
        [0.9841, 0.0105, 0.0054],
        [0.9762, 0.0146, 0.0093],
        [0.8958, 0.0455, 0.0587],
        [0.9771, 0.0141, 0.0088],
        [0.9797, 0.0128, 0.0075],
        [0.9787, 0.0133, 0.0080],
        [0.927

In [17]:
patient_dict = {'gender': 1.0, 'age': 33.0, 'race': 1.0, 'educ': 4.0, 'marry': 1.0, 'house': 2.0, 'pov': 3.28, 'wt': 113.8, 
                'ht': 178.5, 'bmi': 35.7, 'wst': 121.8, 'hip': 113.3, 'dia': 79.0, 'pulse': 73.0, 'sys': 114.0, 'alt': 62.0, 'albumin': 4.0, 
                'ast': 32.0, 'crea': 0.82, 'chol': 170.0, 'tyg': 168.0, 'ggt': 36.0, 'wbc': 6.7, 'hb': 15.5, 'hct': 46.3, 'ldl': 106.0, 
                'hdl': 33.0, 'acratio': 5.24, 'glu': 204.0, 'insulin': 22.77, 'crp': 8.81, 'hb1ac': 9.5, 'mvpa': 840.0, 'ac_week': 0.173077, 
                'dbs': 2.0}


In [19]:
torch.save(best_state_dict, "best_model.pt")


In [18]:
def scoring(patient_dict):
    columns = ['gender', 'age', 'race', 'educ', 'marry', 'house', 'pov', 'wt', 'ht',
           'bmi', 'wst', 'hip', 'dia', 'pulse', 'sys', 'alt', 'albumin', 'ast',
           'crea', 'chol', 'tyg', 'ggt', 'wbc', 'hb', 'hct', 'ldl', 'hdl',
           'acratio', 'glu', 'insulin', 'crp', 'hb1ac', 'mvpa', 'ac_week']
    df_patient = pd.DataFrame([ [patient_dict[col] for col in columns] ], columns=columns)
    X_patient_t = torch.tensor(df_patient.values, dtype=torch.float32)
    model.eval()
    with torch.no_grad():
        output = model(X_patient_t.to(device))  # [1, 3] shape
        prob = torch.softmax(output, dim=1)     # 확률 [a,b,c]
    prob.squeeze(0)
    return (prob[0,1].item()+prob[0,2].item()*2)*50
scoring(patient_dict)


99.99999342074517